In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")
llm.invoke("hi").content

'Hi, I need some help with updating a custom field for one of our users.'

In [5]:

message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

result = llm.invoke(message)
print(result.content)

I'm doing well, thank you! How can I assist you today?


In [6]:

class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
        

In [15]:

bot=Chatbot("you are a helpful assistant")

In [16]:
bot("hi")

"Hi there! 👋\n\nThat's right, I'm here to help! What can I do for you today? 😊  \n\nDo you have a question, need me to write something, or just want to chat?  Let me know!\n"

In [17]:

bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant',
  'content': "Hi there! 👋\n\nThat's right, I'm here to help! What can I do for you today? 😊  \n\nDo you have a question, need me to write something, or just want to chat?  Let me know!\n"}]

In [18]:

bot.execute()

'\n'

In [19]:

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [20]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\91639\AppData\Local\Temp\ipykernel_13748\952804720.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)')


In [21]:

import re

# Compiling the regular expression pattern
action_re = re.compile('^Action: (\w+): (.*)')

# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
C:\Users\91639\AppData\Local\Temp\ipykernel_13748\2705010181.py:4: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)')


In [24]:
# Testing the pattern
match1 = action_re.match(text1)

In [25]:
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [26]:
match1.group(1)

'Move'

In [29]:

match1.group(2)

'North'

In [35]:

match2 = action_re.match(text2)

In [36]:

match2

<re.Match object; span=(0, 18), match='Action: Jump: High'>

In [37]:

match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [31]:
match3 = action_re.match(text3)

In [32]:
match3 

In [33]:
# Extracting the matched groups
if match1:
    print(f"Matched: {match1.group(1)}, {match1.group(2)}")

Matched: Move, North


In [38]:
if match2:
    print(f"Matched: {match2.group(1)}, {match2.group(2)}")  # Jump, High

Matched: Jump, High


In [39]:
if match3:
    print("No match")  # This won't match, as it doesn't start with "Action:"

In [40]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [41]:

import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [42]:
def calculate(number):
    return eval(number)

In [43]:

calculate("2+2")

4

In [44]:

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [45]:

bot = Chatbot(prompt)

In [47]:

next_prompt="Tell me about quantum computing from Wikipedia."

In [48]:

result = bot(next_prompt)

In [49]:

result 

'Thought: I will ask Wikipedia to provide a summary of quantum computing. \n\nAction: wikipedia: Quantum computing\nPAUSE  \n'

In [50]:

actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [51]:
actions

[<re.Match object; span=(0, 36), match='Action: wikipedia: Quantum computing'>]

In [52]:

action, action_input = actions[0].groups()

In [53]:

action_input

'Quantum computing'

In [54]:

action

'wikipedia'

In [57]:
known_actions[action]

<function __main__.wikipedia(query)>

In [58]:
known_actions[action](action_input)

'waves, and <span class="searchmatch">quantum</span> <span class="searchmatch">computing</span> leverages this behavior using specialized hardware. Classical physics cannot explain the operation of these <span class="searchmatch">quantum</span> devices'

In [59]:

def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [60]:
query("hello")

Okay, I understand the loop and my available actions. I'm ready for the first question!  



"Okay, I understand the loop and my available actions. I'm ready for the first question!  \n"

In [61]:

query("how are you?")

Understood. I will follow the Thought-Action-PAUSE-Observation loop to answer your questions. 




'Understood. I will follow the Thought-Action-PAUSE-Observation loop to answer your questions. \n\n'

In [62]:

query("what you doing?")

Okay, I understand the process. I will follow the Thought-Action-PAUSE-Observation loop and use the available actions to find the answer to the question I am asked.  

I'm ready for the first question!  




"Okay, I understand the process. I will follow the Thought-Action-PAUSE-Observation loop and use the available actions to find the answer to the question I am asked.  \n\nI'm ready for the first question!  \n\n"

In [63]:
print(query("Fifteen * twenty five"))

Thought: I need to calculate 15 * 25. 

Action: calculate: 15 * 25
PAUSE 

 -- running calculate 15 * 25
Observation: 375
Answer: 15 * 25 = 375 

Answer: 15 * 25 = 375 



In [66]:
query("Has Simon written about AI?")

Thought: I should check Simon's blog for information about AI.

Action: simon_blog_search: AI 

PAUSE 

 -- running simon_blog_search AI 


ReadTimeout: The read operation timed out